In [1]:
!pip install datasets
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_dataset
from transformers import AutoModelForImageClassification

# ---------------------------
# 1. Caricamento Dataset
# ---------------------------
print("Caricamento dataset Stanford Cars...")
dataset = load_dataset("tanganke/stanford_cars")
print("Dataset caricato con successo!")
print(dataset)

# ---------------------------
# 2. Trasformazioni immagini
# ---------------------------
print("\nPreparazione delle trasformazioni immagini...")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda img: img.convert("RGB")),  # forza RGB
    transforms.ToTensor(),
])
print("Trasformazioni pronte!")

def transform_example(example):
    img = example["image"]
    img = transform(img)
    return {"pixel_values": img, "label": example["label"]}

print("Applicazione delle trasformazioni al dataset (train)...")
train_dataset = dataset["train"].map(transform_example)
print("Train dataset trasformato!")
print("Applicazione delle trasformazioni al dataset (test)...")
test_dataset = dataset["test"].map(transform_example)
print("Test dataset trasformato!")

# Imposta formato PyTorch
train_dataset.set_format(type="torch", columns=["pixel_values", "label"])
test_dataset.set_format(type="torch", columns=["pixel_values", "label"])
print("Formato PyTorch impostato per train e test dataset.")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# ---------------------------
# 3. Modello pre-addestrato
# ---------------------------
device = torch.device("cuda")
print(f"\nDevice in uso: {device}")

print("Caricamento modello pre-addestrato ViT-base...")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

# Sovrascrivi il classificatore finale
model.classifier = nn.Linear(model.classifier.in_features, 196)
model = model.to(device)
print("Modello pronto!")

# ---------------------------
# 4. Training loop semplice con percentuale completamento
# ---------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

def train_model(epochs=1):
    model.train()
    for epoch in range(epochs):
        print(f"\n--- Epoch {epoch+1}/{epochs} ---")
        total_loss = 0
        for i, batch in enumerate(train_loader):
            inputs = batch["pixel_values"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(pixel_values=inputs)
            loss = loss_fn(outputs.logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Percentuale di completamento
            percent_complete = (i + 1) / len(train_loader) * 100
            if (i + 1) % 10 == 0 or (i + 1) == len(train_loader):
                print(f"Batch {i+1}/{len(train_loader)} - {percent_complete:.1f}% completato - Loss media: {total_loss/(i+1):.4f}")

        print(f"Epoch {epoch+1} completata, Loss media: {total_loss/len(train_loader):.4f}")

# Avvio training
print("\nInizio training...")
train_model(epochs=3)
print("Training completato!")

# ---------------------------
# 5. Predizione esempio
# ---------------------------
def predict_one(example):
    model.eval()
    inputs = example["pixel_values"].unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(pixel_values=inputs)
        pred = outputs.logits.argmax(-1).item()
    return pred

sample = test_dataset[0]
print("\nEsempio predizione:")
print("Label reale:", sample["label"])
print("Predizione:", predict_one(sample))


Caricamento dataset Stanford Cars...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset caricato con successo!
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 8144
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 8041
    })
    contrast: Dataset({
        features: ['image', 'label'],
        num_rows: 8041
    })
    gaussian_noise: Dataset({
        features: ['image', 'label'],
        num_rows: 8041
    })
    impulse_noise: Dataset({
        features: ['image', 'label'],
        num_rows: 8041
    })
    jpeg_compression: Dataset({
        features: ['image', 'label'],
        num_rows: 8041
    })
    motion_blur: Dataset({
        features: ['image', 'label'],
        num_rows: 8041
    })
    pixelate: Dataset({
        features: ['image', 'label'],
        num_rows: 8041
    })
    spatter: Dataset({
        features: ['image', 'label'],
        num_rows: 8041
    })
})

Preparazione delle trasformazioni immagini...
Trasformazioni pronte!
Applicazione delle trasformazioni al dat

Map:   0%|          | 0/8144 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
# Avvio training
print("\nInizio training...")
train_model(epochs=1)
print("Training completato!")

In [ ]:
# ---------------------------
# 6. Salvataggio del modello
# ---------------------------
save_path = "vit_cars_model"
model.save_pretrained(save_path)
print(f"Modello salvato in: {save_path}")

Modello salvato in: vit_cars_model


In [ ]:
# ---------------------------
# Creazione mappa indice → nome macchina
# ---------------------------
car_classes = dataset["train"].features["label"].names  # lista dei nomi delle classi

# ---------------------------
# Funzione predizione da URL con nome reale
# ---------------------------
def predict_car_name_from_url(url):
    # Scarica immagine
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")

    # Applica trasformazioni
    img_tensor = transform(img).unsqueeze(0).to(device)

    # Predizione
    model.eval()
    with torch.no_grad():
        outputs = model(pixel_values=img_tensor)
        pred_index = outputs.logits.argmax(-1).item()

    # Recupera nome reale della macchina
    pred_name = car_classes[pred_index]
    return pred_name

# ---------------------------
# Esempio d'uso
# ---------------------------
url = "https://imgs.search.brave.com/uZkgBMtgNYAFn0o3bNvSrgpWBsLBb31lPniLwLSy5o4/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly91cGxv/YWQud2lraW1lZGlh/Lm9yZy93aWtpcGVk/aWEvY29tbW9ucy9h/L2E3LzIwMDhfU3V6/dWtpX1NYNF9Ub3Vy/aW5nLmpwZw"
predizione_nome = predict_car_name_from_url(url)
print("Macchina predetta:", predizione_nome)

Macchina predetta: Honda Odyssey Minivan 2012
